# Part II: A new, parametric type hierarchy

First, let us note that there are two fundamentally different types of agents in
our world: animals and plants. All species such as grass, sheep, wolves, etc.
can be categorized as one of those two.  We can use Julia's powerful,
*parametric* type system to define one large abstract type for all agents
`Agent{S}`. The `Agent` will either be an `Animal` or a `Plant` with a type
parameter `S` which will represent the specific animal/plant
species we are dealing with.

This new type hiearchy can then look like this:


In [1]:
abstract type Species end

abstract type PlantSpecies <: Species end
abstract type Grass <: PlantSpecies end

abstract type AnimalSpecies <: Species end
abstract type Sheep <: AnimalSpecies end
abstract type Wolf <: AnimalSpecies end

abstract type Agent{S<:Species} end

# instead of Symbols we can use an Enum for the sex field
# using an Enum here makes things easier to extend in case you
# need more than just binary sexes and is also more explicit than
# just a boolean
@enum Sex female male

In [27]:
mutable struct World{A<:Agent}
    agents::Dict{Int,A}
    max_id::Int
end

function World(agents::Vector{<:Agent})
    max_id = maximum(a.id for a in agents)
    World(Dict(a.id=>a for a in agents), max_id)
end

# optional: overload Base.show
function Base.show(io::IO, w::World)
    println(io, typeof(w))
    for (_,a) in w.agents
        println(io,"  $a")
    end
end

Now we can create a concrete type Animal with the two parametric types and the fields that we already know from lab 2.

In [2]:
mutable struct Animal{A<:AnimalSpecies} <: Agent{A}
    const id::Int
    energy::Float64
    const Δenergy::Float64
    const reprprob::Float64
    const foodprob::Float64
    const sex::Sex
end

To create an instance of Animal we have to specify the parametric type while constructing it

In [3]:
Animal{Wolf}(1,5,5,1,1,female)
Main.Animal{Main.Wolf}(1, 5.0, 5.0, 1.0, 1.0, Main.female)

Animal{Wolf}(1, 5.0, 5.0, 1.0, 1.0, female)

Note that we now automatically have animals of any species without additional work. Starting with the overload of the show method we can already see that we can abstract away a lot of repetitive work into the type system. We can implement one single show method for all animal species!

<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Implement `Base.show(io::IO, a::Animal)` with a single method for all `Animal`s.
You can get the pretty (unicode) printing of the `Species` types with
another overload like this: `Base.show(io::IO, ::Type{Sheep}) = print(io,"🐑")`

    
</div>

####

<div class="alert alert-block alert-info">
<b>Solution</b>: </div>

####

####

Unfortunately we have lost the convenience of creating plants and animals by simply calling their species constructor. For example, Sheep is just an abstract type that we cannot instantiate. However, we can manually define a new constructor that will give us this convenience back. This is done in exactly the same way as defining a constructor for a concrete type:

In [5]:
#Sheep(id,E,ΔE,pr,pf,s=rand(Sex)) = Animal{Sheep}(id,E,ΔE,pr,pf,s)

Ok, so we have a constructor for Sheep now. But what about all the other billions of species that you want to define in your huge master thesis project of ecosystem simulations? Do you have to write them all by hand? Do not despair! Julia has you covered.

<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Overload all `AnimalSpecies` types with a constructor.
You already know how to write constructors for specific types such as `Sheep`.
Can you manage to sneak in a type variable? Maybe with `Type`?
    
</div>

####

<div class="alert alert-block alert-info">
<b>Solution</b>: </div>

####

In [1]:
# get the per species defaults back


🐑

🐺

####

We have our convenient, high-level behaviour back!

In [15]:
randsex()

female::Sex = 0

In [17]:
Sheep(1)

🐑♂ #1 E=4.0 ΔE=0.2 pr=0.8 pf=0.6

In [18]:
Wolf(2)

🐺♂ #2 E=10.0 ΔE=8.0 pr=0.1 pf=0.2

<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Check the methods for `eat!` and `kill_agent!` which involve `Animal`s and update
their type signatures such that they work for the new type hiearchy.
    
</div>

####

<div class="alert alert-block alert-info">
<b>Solution</b>: </div>

####

####

<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Finally, we can implement the new behaviour for `reproduce!` which we wanted.
Build a function which first finds an animal species of opposite sex and then
lets the two reproduce (same behaviour as before).

    
</div>

####

<div class="alert alert-block alert-info">
<b>Solution</b>: </div>

####

####

In [23]:
s1 = Sheep(1, s=female)

🐑♀ #1 E=4.0 ΔE=0.2 pr=0.8 pf=0.6

In [24]:
s2 = Sheep(2, s=male)

🐑♂ #2 E=4.0 ΔE=0.2 pr=0.8 pf=0.6

In [30]:
w  = World([s1, s2])

World{Animal{🐑}}
  🐑♂ #2 E=4.0 ΔE=0.2 pr=0.8 pf=0.6
  🐑♀ #1 E=4.0 ΔE=0.2 pr=0.8 pf=0.6


In [31]:
reproduce!(s1, w); w

World{Animal{🐑}}
  🐑♂ #2 E=4.0 ΔE=0.2 pr=0.8 pf=0.6
  🐑♀ #3 E=2.0 ΔE=0.2 pr=0.8 pf=0.6
  🐑♀ #1 E=2.0 ΔE=0.2 pr=0.8 pf=0.6


<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Implement the type hiearchy we designed for `Plant`s as well.

</div>

####

<div class="alert alert-block alert-info">
<b>Solution</b>: </div>

####

In [2]:
# constructor for all Plant{<:PlantSpecies} callable as PlantSpecies(...)


In [3]:
# default specific for Grass


####

In [39]:
g = Grass(2)

🌿  #2 10% grown

In [40]:
s = Sheep(3)

🐑♀ #3 E=4.0 ΔE=0.2 pr=0.8 pf=0.6

In [41]:
w = World([g,s])

World{Agent}
  🌿  #2 10% grown
  🐑♀ #3 E=4.0 ΔE=0.2 pr=0.8 pf=0.6


In [42]:
eat!(s,g,w); w

World{Agent}
  🌿  #2 0% grown
  🐑♀ #3 E=4.2 ΔE=0.2 pr=0.8 pf=0.6
